In [1]:
pip install natasha

Note: you may need to restart the kernel to use updated packages.


In [2]:
pip install yargy ipymarkup

Note: you may need to restart the kernel to use updated packages.


In [3]:
pip install ipymarkup

Note: you may need to restart the kernel to use updated packages.


In [4]:
from glob import iglob as list_paths


def load_text(path):
    with open(path) as file:
        return file.read()


texts = sorted(
    load_text(_)
    for _ in list_paths('texts/*.txt')
)

In [5]:
def preprocess(text):
    text = text.replace('\xa0', ' ')
    text = text.replace('\xad', '')
    text = text.translate(DASHES_TRANSLATION)
    return text

In [6]:
print(texts[0])

История mSQL
До 1994 года на рынке были представлены главным образом базы данных, предназначенные для работы со значительными объемами данных и сложными взаимосвязями, которые обладали различными возможностями, но в тоже время требовали значительных вычислительных ресурсов. К числу таких баз данных относятся Oracle, Informix и Sybase. СУРБД с поддержкой SQL и доступных по стоимости в это время не было.

Крупные корпорации и университеты могли себе позволить приобрести мощные вычислительные комплексы, а небольшие организации и пользователи пользовались слабыми настольными базами данных, такими, как Postgres, в которой использовалась разновидность языка QUEL (PostQUEL), но, которая, к сожалению, требовала таких же ресурсов, что и ее аналоги, но не давала преимущества использования SQL в качестве языка запросов.

Дэвид Хьюз
Давид Хьюз aka Bamby часть своей диссертации, которую он писал в Университете Бонд в Австралии, посвятил разработке системы наблюдения и контроля из одной или нескольк

In [7]:
from natasha import (
 Segmenter,
 MorphVocab,
 NewsEmbedding,
 NewsMorphTagger,
 NewsSyntaxParser,
 NewsNERTagger,
 PER,
 ORG,
 NamesExtractor,
 MoneyExtractor,
 AddrExtractor,
 DatesExtractor,
 Doc
)
from ipymarkup import show_box_markup
from ipymarkup.palette import palette, BLUE, RED, GREEN


from yargy import rule, Parser, or_
from yargy.predicates import eq, gram

from yargy.interpretation import fact
from yargy.relations import gnc_relation


In [8]:
from yargy.tokenizer import MorphTokenizer

TOKENIZER = MorphTokenizer()
list(TOKENIZER('Давид Хьюз Университете'))

[MorphToken(
     value='Давид',
     span=[0, 5),
     type='RU',
     forms=[Form('давид', Grams(NOUN,Name,anim,masc,nomn,sing)),
      Form('давида', Grams(NOUN,Name,anim,femn,gent,plur)),
      Form('давида', Grams(NOUN,Name,accs,anim,femn,plur))]
 ),
 MorphToken(
     value='Хьюз',
     span=[6, 10),
     type='RU',
     forms=[Form('хьюз', Grams(NOUN,Sgtm,Surn,anim,masc,nomn,sing))]
 ),
 MorphToken(
     value='Университете',
     span=[11, 23),
     type='RU',
     forms=[Form('университет', Grams(NOUN,inan,loct,masc,sing))]
 )]

In [9]:
emb = NewsEmbedding()
segmenter = Segmenter()
morph_vocab = MorphVocab()
ner_tagger = NewsNERTagger(emb)
names_extractor = NamesExtractor(morph_vocab)
 
text =  'Давид Хьюз aka Bamby часть своей диссертации, которую он писал в Университете Бонд в Австралии, посвятил разработке системы наблюдения и контроля из одной или нескольких точек за группой систем. Главным элементом проекта, который назывался Minerva Network Management System, была база данных для хранения информации обо всех компьютерах в сети. Изобретателем MySQL является Михаил Видениус aka Monty из шведской компании TcX. В 1979 году он разработал средство управления базами данных, которое называлось UNIREG.'  #текст добавляем сюда

doc = Doc(texts[0])

doc.segment(segmenter)
 
doc.tag_ner(ner_tagger)

for span in doc.spans:
    span.normalize(morph_vocab)
   
{_.text: _.normal for _ in doc.spans}

#{_.normal: _.fact.as_dict for _ in doc.spans if _.fact}

show_box_markup(texts[0], doc.spans, palette=palette(PER=BLUE, ORG=RED, LOC=GREEN))

In [10]:
for span in doc.spans:
    if span.type == PER:
        span.extract_fact(names_extractor)

#{_.normal for _ in doc.spans if _.type == PER}    
{_.normal: _.fact.as_dict for _ in doc.spans if _.fact}

{'Дэвид Хьюз Давид Хьюз': {'first': 'Дэвид', 'last': 'Хьюз'},
 'Хьюз': {'last': 'Хьюз'},
 'Хьюза': {'last': 'Хьюза'},
 'Михаил Видениус': {'first': 'Михаил', 'last': 'Видениус'},
 'Видениус': {'last': 'Видениус'},
 'Хьюзом': {'last': 'Хьюзом'},
 'Дэвид Оксмарк': {'first': 'Дэвид', 'last': 'Оксмарк'},
 'Дэвид': {'first': 'Дэвид'},
 'Рэнди Яргера': {'first': 'Рэнди', 'last': 'Яргера'},
 'Джорджа Риза': {'first': 'Джорджа', 'last': 'Риза'},
 'Тима Кинга': {'first': 'Тима', 'last': 'Кинга'}}

In [11]:
for span in doc.spans:
    if span.type == ORG:
        span.extract_fact(names_extractor)
    
{_.normal for _ in doc.spans if _.type == ORG}

{'API',
 'Detron HB',
 'Informix',
 'Minerva',
 'MySQL',
 'Oracle',
 'PostQUEL',
 'Postgres',
 'Sybase',
 'TcX',
 'UNIREG',
 'mSQL',
 'Интернет',
 'РНР',
 'СУРБД',
 'ТсХ',
 'Университете Бонд'}

In [12]:
{_.text: _.normal for _ in doc.spans if _.type == ORG}

{'Oracle': 'Oracle',
 'Informix': 'Informix',
 'Sybase': 'Sybase',
 'Университете Бонд': 'Университете Бонд',
 'Minerva': 'Minerva',
 'Postgres': 'Postgres',
 'PostQUEL': 'PostQUEL',
 'СУРБД': 'СУРБД',
 'mSQL': 'mSQL',
 'TcX': 'TcX',
 'UNIREG': 'UNIREG',
 'API': 'API',
 'ТсХ': 'ТсХ',
 'Интернет': 'Интернет',
 'MySQL': 'MySQL',
 'Detron HB': 'Detron HB',
 'РНР': 'РНР'}

In [13]:

gnc = gnc_relation()

Name = fact(
    'Name',
    ['first', 'last']
)
FIRST = gram('Name').interpretation(
    Name.first.inflected()
).match(gnc)

LAST = gram('Surn').interpretation(
    Name.last.inflected()
).match(gnc)
NAME = or_(
    rule(
        FIRST,
        LAST
    ),
    rule(
        LAST,
        FIRST
    )
).interpretation(
    Name
)


#NAME = rule(eq('История'))
parser = Parser (NAME)

    
matches = list(parser.findall(texts[0]))
spans = [_.span for _ in matches]
show_box_markup(texts[0], spans)



In [14]:
#вывод имен в ИМ.П
for match in parser.findall(texts[0]):
    print(match.fact) 

Name(first='дэвид', last='хьюз')
Name(first='давид', last='хьюз')
Name(first='михаил', last='видениус')
Name(first='дэвид', last='оксмарк')
Name(first='тим', last='кинг')


In [15]:
#doc.ner.print()
print(doc.spans)
#print(doc.tokens)
#print(doc.sents[:5])

[DocSpan(start=310, stop=316, type='ORG', text='Oracle', tokens=[...], normal='Oracle'), DocSpan(start=318, stop=326, type='ORG', text='Informix', tokens=[...], normal='Informix'), DocSpan(start=329, stop=335, type='ORG', text='Sybase', tokens=[...], normal='Sybase'), DocSpan(start=822, stop=843, type='PER', text='Дэвид Хьюз\nДавид Хьюз', tokens=[...], normal='Дэвид Хьюз Давид Хьюз', fact=DocFact(slots=[...])), DocSpan(start=898, stop=915, type='ORG', text='Университете Бонд', tokens=[...], normal='Университете Бонд', fact=DocFact(slots=[...])), DocSpan(start=918, stop=927, type='LOC', text='Австралии', tokens=[...], normal='Австралии'), DocSpan(start=1202, stop=1206, type='PER', text='Хьюз', tokens=[...], normal='Хьюз', fact=DocFact(slots=[...])), DocSpan(start=1309, stop=1316, type='ORG', text='Minerva', tokens=[...], normal='Minerva'), DocSpan(start=1496, stop=1500, type='PER', text='Хьюз', tokens=[...], normal='Хьюз', fact=DocFact(slots=[...])), DocSpan(start=1774, stop=1782, type=

In [16]:
spans = [(833, 843, 'PER'),(898, 915, 'ORG'),(2767, 2782, 'PER'),(2793, 2817, 'ORG'),(4554, 4567, 'PER'),(4591, 4600, 'ORG')]
show_box_markup(texts[0], spans)